# Web App Targeted Image Generation - [Github link](https://github.com/yunqing-me/AttackVLM)
## Zhao, Y., Pang, T., Du, C., Yang, X., Li, C., Cheung, N.-M., & Lin, M. (2023). *On evaluating adversarial robustness of large vision-language models.* Advances in Neural Information Processing Systems (NeurIPS 2023).
Elias El Khoury - 202301237 - eje11

## Installing Flask Dependencies to Connect to Google Colab via API

In [1]:
!pip install flask_cors

In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import base64
import io
from PIL import Image
import threading

In [3]:
app = Flask(__name__)
CORS(app)  # Enable CORS for web requests

## Installing Necessary Dependencies for Target Image Generation Function: Drive, Githubs, Libraries
This notebook modifies the Target Image Generation notebook by removing the access to the google drive.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The drive contains the downloaded MS-COCO text file with the captions.

In [5]:
import os
data_dir = '/content/drive/MyDrive/EECE-608/Project/WebApp/webtxt2img_coco.py'

In [6]:
!git clone https://github.com/yunqing-me/AttackVLM.git

fatal: destination path 'AttackVLM' already exists and is not an empty directory.


In [7]:
!pip install lmdb tqdm wandb torchvision

In [8]:
!git clone https://github.com/CompVis/stable-diffusion.git

fatal: destination path 'stable-diffusion' already exists and is not an empty directory.


In [9]:
%cd stable-diffusion

/content/stable-diffusion


In [10]:
# Move the txt2img_coco.py file using the correct variable referencing
!cp /content/drive/MyDrive/EECE-608/Project/WebApp/webtxt2img_coco.py /content/stable-diffusion

In [11]:
!pip install omegaconf
!pip install imwatermark
!pip install einops
!pip install xformers
!pip install openai
!pip install lmdb
!pip install scipy
!pip install ftfy
!pip install regex

In [12]:
!pip install invisible-watermark # Solution from: https://github.com/CompVis/stable-diffusion/issues/332

In [13]:
!pip install pytorch_lightning

In [14]:
!pip install --upgrade pytorch-lightning
!pip install lightning
from lightning.pytorch.utilities import rank_zero_only

Note when running the next code section:


*   Replace in '/stable-diffusion/Idm/models/diffusion/ddpm.py' line 19 with: "from pytorch_lightning.utilities.rank_zero import rank_zero_only". (source: https://stackoverflow.com/questions/74289972/cannot-import-name-rank-zero-only-from-pytorch-lightning-utilities-distribute)



In [15]:
!pip install taming-transformers

In [16]:
!pip install git+https://github.com/CompVis/taming-transformers.git

  Cloning https://github.com/CompVis/taming-transformers.git to /tmp/pip-req-build-09m4sjq3
  Running command git clone --filter=blob:none --quiet https://github.com/CompVis/taming-transformers.git /tmp/pip-req-build-09m4sjq3
  Resolved https://github.com/CompVis/taming-transformers.git to commit 3ba01b241669f5ade541ce990f7650a3b8f65318
  Preparing metadata (setup.py) ... done


ImportError: cannot import name 'VectorQuantizer2' from 'taming.modules.vqvae.quantize' (/usr/local/lib/python3.11/dist-packages/taming/modules/vqvae/quantize.py)

fix source: https://github.com/CompVis/stable-diffusion/issues/72

Download the Correct File

In [17]:
!wget https://raw.githubusercontent.com/CompVis/taming-transformers/master/taming/modules/vqvae/quantize.py -O quantize.py

--2025-03-31 12:17:01--  https://raw.githubusercontent.com/CompVis/taming-transformers/master/taming/modules/vqvae/quantize.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18182 (18K) [text/plain]
Saving to: ‘quantize.py’

quantize.py         100%[===================>]  17.76K  --.-KB/s    in 0.001s  

2025-03-31 12:17:02 (23.4 MB/s) - ‘quantize.py’ saved [18182/18182]



Replace the Old quantize.py

In [18]:
!mv quantize.py /usr/local/lib/python3.11/dist-packages/taming/modules/vqvae/quantize.py

In [19]:
from taming.modules.vqvae.quantize import VectorQuantizer2
print("VectorQuantizer2 is successfully imported.")

VectorQuantizer2 is successfully imported.


In [20]:
!pip install clip
!pip install kornia

## Generating Targeted Images using Captions

changed n_samples from 2 to 1 because processing only 1 caption at a time in this prototype

In [21]:
# Function that uses your existing code to generate an image
def generate_image_from_caption(input_caption):
    !python /content/stable-diffusion/webtxt2img_coco.py \
    --ddim_eta 0.0 \
    --n_samples 1 \
    --n_iter 5 \
    --scale 7.5 \
    --ddim_steps 50 \
    --plms \
    --skip_grid \
    --ckpt '/content/drive/MyDrive/EECE-608/Project/Paper1/_model_pool/sd-v1-4-full-ema.ckpt' \
    --caption "{input_caption}" \
    --outdir '/content/drive/MyDrive/EECE-608/Project/WebApp/target_image_test/'
    pass

debugging and testing the python command file

In [22]:
#caption_test = "This little girl is taking tennis lessons to learn how to play."

In [23]:
#generate_image_from_caption(caption_test)

2025-03-31 12:17:26.882542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743423447.046611   25844 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743423447.087574   25844 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
Seed set to 42
Loading model from /content/drive/MyDrive/EECE-608/Project/Paper1/_model_pool/sd-v1-4-full-ema.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-predicti

connecting to flask server

In [24]:
@app.route('/generate', methods=['POST'])
def generate():
    data = request.json
    caption = data.get('caption')

    if not caption:
        return jsonify({'error': 'Caption is required'}), 400

    try:
        # Generate image using your existing code
        generate_image_from_caption(caption)
        # Get filename path to the most recent generated image
        import glob
        files = sorted(glob.glob("/content/drive/MyDrive/EECE-608/Project/WebApp/target_image_test/samples/*.png"), key=os.path.getmtime, reverse=True)
        filename = files[0]

        with open(filename, "rb") as f:
          encoded = base64.b64encode(f.read()).decode("utf-8")

        # Return to frontend
        return jsonify({
            'success': True,
            'image': f"data:image/png;base64,{encoded}"
        })
        """
        # Convert image to base64 for sending over HTTP
        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode('utf-8')

        return jsonify({
            'success': True,
            'image': f'data:image/png;base64,{img_str}'
        })
        """
    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Run the Flask app
threading.Thread(target=lambda: app.run(host='0.0.0.0', port=8000, debug=True, use_reloader=False)).start()

print("API server running at http://localhost:8000")

API server running at http://localhost:8000
 * Serving Flask app '__main__'
 * Debug mode: on


ngrok tunnels port 8000 where flask server is running to the public URL generated

In [25]:
!pip install pyngrok
from pyngrok import ngrok

# Authenticate with your authtoken
ngrok.set_auth_token("2v4LdF2hH6ksqmuq8D7wRK5TtZg_5vU9J2siWFTG1UCV8M9pJ")
# Start ngrok tunnel to expose the API
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit


Public URL: https://af03-34-125-163-86.ngrok-free.app


use the public URL to change the previously used one in image-generation.ts file